# 새 데이터 가져오기

In [30]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd

In [31]:
head = {'User-Agent':
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0'}
data = []
with requests.Session() as s:
    for page in range(1,3):
        url = f"https://www.saramin.co.kr/zf_user/jobs/list/job-category?cat_mcls=2&page={page}&page_count=1000&sort=RD"
        r = s.get(url, headers=head)
        data.append(r)
        print(page)
        if r.status_code != 200:
            break


1
2


In [32]:
item_list = []
for r in data:
    soup = BeautifulSoup(r.text)
    box_item_list = soup.select('div.box_item')


    for box_item in box_item_list:
        temp = {}
        temp['company_name'] = box_item.select_one('.company_nm > .str_tit').text.strip()
        temp['company_sn'] = box_item.select_one('.interested_corp')['csn']
        temp['is_headhunting'] = '헤드헌팅' in box_item.select_one(".company_nm").text
        temp['job_idx'] = int(box_item.find('button', attrs={'rec_idx': True})['rec_idx'])
        temp['job_title'] = box_item.select_one('.notification_info a.str_tit')['title']
        temp['job_link'] = box_item.select_one('.notification_info a.str_tit')['href']
        temp['job_sectors'] = [i.text for i in box_item.select('.notification_info .job_meta .job_sector span')]
        work_place_element = box_item.select_one('.recruit_info .work_place')
        temp['work_place'] = work_place_element.text if work_place_element is not None else None
        temp['career_info'] = box_item.select_one('.recruit_info .career').text
        temp['education_info'] = box_item.select_one('.recruit_info .education').text
        temp['support_date'] = box_item.select_one('.support_info .date').text
        temp['deadline'] = box_item.select_one('.support_info .deadlines').text
        item_list.append(temp)


# 새 시트 만들기

In [35]:
df_new = pd.DataFrame(item_list)
df_new["update"] = pd.to_datetime("now")
df_new["job_link"] = df_new.job_link.apply(lambda x: f"https://www.saramin.co.kr{x}")


def process_deadline(row):
    x = row["deadline"]

    word = x.split(" ")[0]

    if "일" in word:
        delta = pd.Timedelta(days=int(word.split("일")[0]))
    elif "시간" in word:
        delta = pd.Timedelta(hours=int(word.split("시간")[0]))
    elif "분" in word:
        delta = pd.Timedelta(minutes=int(word.split("분")[0]))

    if "등록" in x:
        row["resistered"] = row["update"] - delta
    elif "수정" in x:
        row["modified"] = row["update"] - delta
    else:
        pass

    return row


df_new = df_new.apply(process_deadline, axis=1)

df_new.set_index("job_idx", inplace=True)


# eda
## 사명 변경된 적 있는지
```
final_df = df[df.groupby('company_sn')['company_name'].transform('nunique') > 1].sort_values(by='job_idx')[['company_sn', 'company_name']].drop_duplicates()
final_df['name_count'] = final_df.groupby('company_sn').cumcount() + 1
final_df.pivot_table(columns='name_count', index='company_sn', values='company_name', aggfunc='first')
```

## 목록에 표시된 해시태그
- 하지만 각 공고마다 가지고 있는 태그 전체 내용이 아니다.
```
df2 = df[['job_idx','job_sectors']]
df2.explode('job_sectors').groupby('job_sectors').size().sort_values(ascending=False).to_csv('job_sectors.csv')
df.explode('job_sectors').groupby('job_sectors').get_group('크롤링')

## jobsector 데이터가 조회할때마다 달라지는지 확인 필요
- 다섯개 이상 선택해도 리스트에 표시되지 않음
- 지정된 우선순위 순서로 표시되는지 아니면 조회할때마다 무작위로 5개 선택하는지 확인 필요

# 구글 시트에서 이전 시트 가져오기

In [ ]:
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe

gc = gspread.service_account()

sh=gc.open_by_key('1vkZ7FF1bgACwXKxkmSNemmSNiti4Bd2M177w-8H80ww').sheet1




# modified          datetime64[ns]
# resistered        datetime64[ns]
# update            datetime64[ns]
# df_old의 세가지 열 데이터타입 변경


df_old = get_as_dataframe(sh, evaluate_formulas=False)
df_old.dropna(subset=['job_idx'], inplace=True)
df_old['job_idx'] = df_old['job_idx'].astype(int)
df_old.set_index('job_idx', inplace=True)
df_old['is_headhunting'] = df_old['is_headhunting'].astype(bool)
df_old["modified"] = pd.to_datetime(df_old["modified"])
df_old["resistered"] = pd.to_datetime(df_old["resistered"])
df_old["update"] = pd.to_datetime(df_old["update"])
df_old

# 새시트, 이전시트 병합
- jobsector 열 리스트 확장
- 제목 변경되는 경우 특이 케이스로 보고 따로 표시
- 수정일 : 새 시트 우선
- 작성일 : 이전 시트 우선, 원래 없다가 새로 생겨나는 경우 없을듯
- update 열 : 마지막으로 조회한 시간, 새 시트 우선
- 어떻게 합치는게 세련된 방법인지 모르겠다

- old에서 안쓰는 열은 드롭? 안돼 그러면 아우터가 아니니까

- 작성일 관측 불가면 어떻게 할지?

In [39]:
combined_df = df_new.combine_first(df_old)


In [40]:
# resistered가 비어있는 경우, modified로 채우기
combined_df["resistered"] = combined_df["resistered"].combine_first(combined_df["modified"])

In [43]:
assert combined_df.shape[1] == 14, "The number of columns is not 14."
print("The number of columns is 14.")

# Merge the dataframes
merged_df = pd.merge(df_old, df_new, how='outer', on=['job_idx'])

# Apply conditions to overlapping parts
# modified new 우선, resistered old 우선, update new 우선

merged_df["resistered"] = merged_df["resistered_x"].fillna(merged_df["resistered_y"])
merged_df["modified"] = merged_df["modified_x"].fillna(merged_df["modified_y"])
merged_df["update"] = merged_df["update_y"].fillna(merged_df["update_x"])

# Drop the columns
merged_df = merged_df.drop(columns=["resistered_x", "resistered_y", "modified_x", "modified_y", "update_x", "update_y"])
merged_df

# 새 시트로 덮어쓰기

In [44]:
# Clear the contents of the sheet
sh.clear()

# Set the new dataframe in the sheet
set_with_dataframe(sh, combined_df, include_index=True, allow_formulas=False)
